# Module Attendance

Student ID: B721581 | Data Completed: 22/01/23

The following is a Python program that produces a table of the attendance of a module for a particular week. Given the module code and week number, the program:
* Reads the relevant data from the SQL database
* Displays a table of the attendance of each session of that module in that week.

**The validity checks for this task are built into the GUI menu**

## Imports

In [1]:
import pandas as pd
import numpy as np
import sqlite3

## Functions

In [2]:
def get_module_att_data(module_code):
    """
    Gets the sessions and attendance data for a 
    given module from the SQL database.
    
    input: module code
    output: returns a data dictionary of sessions and attendance data 
            & module code
    
    db_name = name of database
    data = data dictionary of sessions and attendance data
    module = module_code = module code
    conn = connection to SQL database
    table_list = list of tables in the SQL database
    """
    
    # assign local variables and collections
    db_name = 'CWDatabase'
    data = {}
    module = module_code.strip().upper()
    

    ## read data from SQL database
    
    conn = sqlite3.connect(db_name + '.db')

    # get list of tables
    sqlstm = 'SELECT name FROM sqlite_master WHERE type = "table"'
    table_list = pd.read_sql(sqlstm, conn)
    table_list = list(table_list['name'])

    # loop through the list of tables and read in relevant data tables
    for table in table_list:
        if table[:6] == module:
            data[table] = pd.read_sql('SELECT * FROM %s' %(table), conn)
            data[table] = data[table].drop(columns = 'index')
        else:
            continue

    conn.close()
    
    return (data, module)



def module_att_table(data, module, week_no):
    """
    Displays a table of the attendance of a module for a particular week.

    input: data dictionary of sessions and attendance data
           & module code
           & week number
    output: returns the formatted table
    
    data = data dictionary of sessions and attendance data
    module = module_code = module code
    week_number = week_no = week number
    module_att = dataframe of module attendance
    module_att_values = dictionary of attendance % values for each week
    perc_values = list of attendance % values for given week
    att_format = specified format for attendance % column in table
    table_title = title string for the table caption
    color_code() = function used for conditional formatting
    """
    
    # assign local variables and collections
    week_number = int(week_no)
    module_att = pd.DataFrame()
    perc_values = []
    att_format = {'Attendance %':'{:.1f}'}
    table_title = 'Attendance Record for %s Week %s:' %(module, week_number)

    # create smaller dataframe from sessions data
    for name, df in data.items():
        if name[7:10] == 'ses':
            data[name] = data[name][['week', 'start_time', 'room', 'type']]
            module_att = pd.DataFrame(data[name])
        else:
            continue
        
    # get dictionary of attendance % values for each week
    for name, df in data.items():
        if name[7:10] == 'att':
            module_att_values = dict(df.mean(axis = 0) * 100)
           
            # get list of attendance % values for given week
            for k, v in module_att_values.items():
                if k[4:5] == str(week_number):
                    perc_values.append(v)
                else:
                    continue
        else:
            continue


    # create smaller dataframe of just given week and add attendance % values
    module_att.set_index('week', inplace = True)
    module_att = module_att.loc[['W' + str(week_number)]]
    module_att['Attendance %'] = perc_values


    def color_code(value):
        '''
        Conditionally formats the 'value' passed into the function.

        input: value or subset of values to be formatted
        output: formatted background colour
        
        value = value or subset of values
        lower_limit = specified lower limit
        upper_limit = specified upper limit
        colour = background colour
        '''
        lower_limit = 50
        upper_limit = 70

        if value < lower_limit:
            colour = 'tomato'
        elif value > upper_limit:
            colour = 'lightgreen'
        else:
            colour = 'yellow'
            
        return 'background-color: %s' % colour


    # clean up dataframe for formatting  
    module_att.reset_index(inplace = True)
    module_att.drop(axis = 1, columns = 'week', inplace = True)
    module_att = module_att.rename(columns = {'start_time':'Time', 
                                              'room':'Room',
                                              'type': 'Type'})

    # format the dataframe 
    module_att = module_att.style\
                           .format(att_format)\
                           .applymap(color_code, subset = 'Attendance %')\
                           .set_caption(table_title)\
                           .set_table_styles([{'selector': 'caption',
                                               'props': [('font-size', 
                                                          '18px')]}])\
                           .hide_index()

    display(module_att)

## Main Function

In [3]:
def get_module_att_table(module_code, week_no):
    """
    Combines the above functions into one main function:
        - Reads the data from the SQL database
          and creates the table
    
    input: module code
           & week number
    output: displays the formatted table
    
    module = module_code = module code
    week_no = week number
    data = data dictionary of sessions and attendance data
    """
    
    data, module = get_module_att_data(module_code)
    
    return module_att_table(data, module, week_no)

## Function Demo

In [4]:
get_module_att_table('COA111', 1)

Time,Room,Type,Attendance %
09:00,U020,Lecture,73.3
10:00,SMB014,Lecture,65.6
